<a href="https://colab.research.google.com/github/yodakaz/ASMK_yoda/blob/main/PKL%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E7%A2%BA%E8%AA%8D%E3%83%BB%E4%BD%9C%E6%88%90%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.chdir('/content')
!git clone https://github.com/MING410/local #データセット

Cloning into 'local'...
remote: Enumerating objects: 5839, done.
remote: Counting objects: 100% (1934/1934), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 5839 (delta 1799), reused 1934 (delta 1799), pack-reused 3905
Receiving objects: 100% (5839/5839), 468.04 MiB | 26.35 MiB/s, done.
Resolving deltas: 100% (3689/3689), done.
Checking out files: 100% (7539/7539), done.


In [5]:
#作成したpklファイル確認用
import pickle
#with open("retrieval-SfM-120k.pkl", mode="rb") as f:
with open("mitsubishi_dataset.pkl", mode="rb") as f:
  hoge = pickle.load(f)#['train']
  print(hoge.keys()) #キーの取得
  #print(hoge['cids'])
  for ho in hoge['cids']:
    if '.DS_Store' in ho:
      print(ho)
  print(len(hoge['cids']))
  print(len(hoge['qidxs']), max(hoge['qidxs']))
  print(len(hoge['pidxs']), max(hoge['pidxs']))
  print(len(hoge['cluster']), max(hoge['cluster']))

#with open("retrieval-SfM-120k-val-eccv2020.pkl", mode="rb") as f:
with open("mitsubishi_dataset_val-eccv2020.pkl", mode="rb") as f:
  hoge = pickle.load(f)
  print(hoge.keys()) #キーの取得
  #print(hoge['cids'])
  for ho in hoge['cids']:
    if '.DS_Store' in ho:
      print(ho)
  print(len(hoge['cids']))
  print(len(hoge['qidx']), max(hoge['qidx']))
  print(len(hoge['gnd']))#, max(hoge['pidxs']))
  print(len(hoge['cluster']), max(hoge['cluster']))

dict_keys(['cids', 'qidxs', 'pidxs', 'cluster'])
1035
690 1035
690 1035
1035 49
dict_keys(['cids', 'qidx', 'gnd', 'cluster'])
657
239 22
239
657 49


In [4]:
#for 学習/学習結果の評価用
import os
import random
import pickle
import numpy as np

"""
画像読み込み→パスの格納('cids')，インデックスの分類('qidx', 'gnd')
              部品名の分類('cluster')
make_q_gndのparameter
    fanemes：部品ごとの画像
    n：前後の部品の画像とインデックスを合わせる変数
    (d：randomで抽出しているので使用していない)
    obj_2：クエリーのインデックスを格納するリスト
    obj_3：ポジティブのインデックスを格納するリスト
"""
def make_qp(folder, fnames, n, c0, d, obj_1, obj_2, obj_3, obj_4):
  if len(fnames)%d == 0:
    for i in range(len(fnames)-d):
      obj_1.append("{}/{}".format(folder, fnames[i]))
      obj_4.append(c0)
      if i%d == d-1:
        obj_2.append(i+n)
      elif i%d == 0:
        obj_3.append(i+n)
  
  else:#if len(fnames)%d != 0:
    for i in range(len(fnames)-len(fnames)%d-d):
      obj_1.append("{}/{}".format(folder, fnames[i]))
      obj_4.append(c0)
      if i%d == d-1:
        obj_2.append(i+n)
      elif i%d == 0:
        obj_3.append(i+n)
  return obj_1, obj_2, obj_3, obj_4

def make_q_gnd(folder, fnames, n, c0, obj_1, obj_2, obj_3, obj_4):
  for j in range(len(fnames)):
    obj_1.append("{}/{}".format(folder, fnames[j]))
    obj_4.append(c0)

  qidx, ok, junk = [], [], []
  idx = [i for i in range(len(fnames))]
  #print(idx)
  #breakpoint()
  if len(idx) > 5:
    qidxs = random.sample(idx, 5)
  else:
    qidxs = random.sample(idx, len(idx))
  qidxs.sort()
  for qidx in qidxs:
    obj_2.append(qidx)

  for i in range(len(qidxs)):
  #for i in range(5):
    if len(idx) <= 20:
      random_max = len(idx)
    else:
      random_max = 20
    ok = random.sample(idx, random.randint(1,random_max))
    ok.sort()
    junk = idx.copy()
    for j in ok:
      junk.remove(j)
    obj_3.append({'ok':np.array(ok), 'junk':np.array(junk)})

  return obj_1, obj_2, obj_3, obj_4

root_path = '/content/local/mitsubishi_dataset/png_images'
os.chdir(root_path)
obj_t1, obj_t2 ,obj_t3, obj_t4 = [], [], [], []
obj_e1, obj_e2 ,obj_e3, obj_e4 = [], [], [], []

nt0, ne0, c0 = 0, 0, 0
for folder in os.listdir(root_path):
  nt = nt0+1
  ne = ne0+1
  c0 = c0+1
  fnames = []
  #for i in range(len(os.listdir(folder))):
  for fname in os.listdir(folder):
    #fname = os.listdir(folder)[i]
    if fname[-4:] == '.png':
      #obj_1.append("{}/{}".format(folder, fname)) #filename of images
      #obj_4.append(c0) #cluster (same number ; 2D has 1 camera position)
      fnames.append(fname)
    #print(fnames)
  fnames_t = fnames[:int(len(fnames)/3)*2]
  fnames_e = fnames[int(len(fnames)/3)*2:]

    #for d in range(5,6):
  obj_t1, obj_t2 ,obj_t3, obj_t4 = make_qp(folder, fnames_t, nt, c0, 3, obj_t1, obj_t2 ,obj_t3, obj_t4)
  obj_e1, obj_e2 ,obj_e3, obj_e4 = make_q_gnd(folder, fnames_e, ne, c0, obj_e1, obj_e2 ,obj_e3, obj_e4)
  nt0, ne0 = len(obj_t1), len(obj_e1)      

obj_t = {'cids':obj_t1, 'qidxs':obj_t2+obj_t3, 'pidxs':obj_t3+obj_t2, 'cluster':obj_t4}
obj_e = {'cids':obj_e1, 'qidx':obj_e2, 'gnd':obj_e3, 'cluster':obj_e4}

#print(obj)
os.chdir('/content')
with open("mitsubishi_dataset.pkl", mode="wb") as ft:
    #pickle.dump(, f)
    pickle.dump(obj_t, ft)

with open("mitsubishi_dataset_val-eccv2020.pkl", mode="wb") as fe:
    #pickle.dump(, f)
    pickle.dump(obj_e, fe)

In [ ]:
import pickle

#with open("mitsubishi_dataset_val-eccv2020.pkl", mode="rb") as f:
with open("retrieval-SfM-120k-val-eccv2020.pkl", mode="rb") as f:
    hoge = pickle.load(f)
    print(hoge.keys())
    print(hoge['cids'][:2])
    print(hoge['qidx'][:2])
    print(hoge['gnd'][:0])
    print(hoge['cluster'][:2])
    """
    print(len(hoge['cids']))
    print(len(hoge['qidx']))
    print(len(hoge['gnd']), hoge['gnd'][:10])
    print(len(hoge['cluster']), hoge['cluster'][820:840])
    print(max(hoge['cluster']))
    """
    """
    aa, bb, cc = [], [], []
    for i in range(len(hoge['gnd'])):
      a, b = hoge['gnd'][i]['ok'], hoge['gnd'][i]['junk']
      c = hoge['qidx']
      aa.append(a)
      bb.append(b)
      #cc.append(c)
      #c = max(hoge['gnd'][i]['junk'])

      #print(a, a+b, c)
      #print('ok',len(hoge['gnd'][i]['ok']), max(hoge['gnd'][i]['ok']))
      #print('junk',len(hoge['gnd'][i]['junk']), max(hoge['gnd'][i]['junk']))
    #print(cc)
    #print(len(set(cc[2]) & set(cc[1])))
    for i in range(len(aa)):
      d = set(bb[i]) & set(c)
      print(len(d), d)
    """

dict_keys(['cids', 'qidx', 'gnd', 'cluster'])
['0028a0f1259f877a22c2981b22504265', '006d49ca8bf31c0eff72bc789ccb9d21']
[364  80]
[]
[0 0]


In [ ]:
import pickle

with open("retrieval-SfM-120k.pkl", mode="rb") as f:
    hoge = pickle.load(f)
    print(hoge['train'].keys())
    print(len(hoge['train']['cids']))#, hoge['train']['cids'])
    """
    print(len(hoge['train']['qidxs']))
    print(len(hoge['train']['pidxs']))
    print(len(hoge['train']['cluster']))

    print(hoge['train']['cids'][-10:])
    print(min(hoge['train']['qidxs']))
    print(min(hoge['train']['pidxs']))
    print(hoge['train']['cluster'][-10:])
    """

dict_keys(['cids', 'qidxs', 'pidxs', 'cluster'])
91642


In [ ]:
import pickle

with open("retrieval-SfM-120k.pkl", mode="rb") as f1:
    hoge1 = pickle.load(f1)
    print(hoge1['train'].keys())

with open("retrieval-SfM-120k-val-eccv2020.pkl", mode="rb") as f2:
    hoge2 = pickle.load(f2)
    print(hoge2.keys())

cids1, cids2 = hoge1['train']['cids'], hoge2['cids']
print(len(cids1), len(cids2))
print(cids2[:10])
samecid = set(cids1) & set(cids2)
print(len(list(samecid)))

qidxs1, qidxs2 = hoge1['train']['qidxs'], hoge2['qidx']
print(len(qidxs1), len(qidxs2))
sameqidx = set(qidxs1) & set(qidxs2)
print(len(list(sameqidx))) #重複はあまり関係ない

dict_keys(['cids', 'qidxs', 'pidxs', 'cluster'])
dict_keys(['cids', 'qidx', 'gnd', 'cluster'])
91642 12431
['0028a0f1259f877a22c2981b22504265', '006d49ca8bf31c0eff72bc789ccb9d21', '00819fa357902d32d5d5cc5ac709f83c', '0091e85c350ce634407698b7e86bb779', '013764059c49cc1db4948eb7b1bd0a7e', '01381870050b91cd45f698b8fa35aee6', '01c8fc943f189a76a49ea22c2d8b1446', '01d88cd0ad430d51c64f0c6d2eaa328e', '01f3920b622bdcc17fb238d62062f55d', '0222fe3dfb5ff234ded8d1e980abef6d']
0
181697 719
682


In [ ]:
idx = [i for i in range(10)]
ok = [2,4,7]
print(idx, ok)
out = idx.copy()
for i in ok:
  out.remove(i)
print(out)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9] [2, 4, 7]
[0, 1, 3, 5, 6, 8, 9]
